In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import os 
from datetime import datetime, timedelta

# Load the input data
data = pd.read_csv(r'C:\Users\dell\Desktop\DataPrime\FULL_DATAPRIME_DATA.CSV')




In [2]:
# Rename columns
data = data.rename(columns={"price": "consider", "kawasan": "scheme", "district": "district_to", "state": "state_to"})



In [3]:
# Check for missing values
missing_values = data.isnull().sum()

# Print the number of missing values in each column
print(missing_values)

id                       0
state_to                 0
district_to           1418
mukim                 1340
section             526990
scheme                1188
address                 48
lot_type                16
lot_no                   5
status               49234
transaction_date         0
landarea                 0
unit                     5
title_type              15
title_no                36
sector                   0
tenure                   0
leasehold_period    414450
property_type         3967
floor_above           2118
floor_below          49234
main_floor_area      47553
consider                 0
dtype: int64


In [4]:

#To Create teh Duration_days table

data['transaction_date'] = pd.to_datetime(data['transaction_date'])


current_date = datetime.now()
data['duration_days'] = (current_date - data['transaction_date']).dt.days


In [5]:


# Calculate mean value for each district_to, excluding NaN, 0, and blank values
data_mean = data[data['main_floor_area'].notnull() & (data['main_floor_area'] != 0) & (data['main_floor_area'] != '')].groupby('district_to')['main_floor_area'].mean()


# Replace only the NaN values with the corresponding district_to mean value
data['main_floor_area'] = np.where(data['main_floor_area'].isnull(), data['district_to'].map(data_mean), data['main_floor_area'])


# Replace only the "0" values with the corresponding district_to mean value
data['main_floor_area'] = np.where(data['main_floor_area'] == 0, data['district_to'].map(data_mean), data['main_floor_area'])



# Calculate mean value for each state_to, excluding NaN and blank values
state_mean = data[data['main_floor_area'].notnull() & (data['main_floor_area'] != '')].groupby('state_to')['main_floor_area'].mean()


# Replace only the NaN and blank values with the corresponding state_to mean value
data.loc[data['main_floor_area'].isnull() | (data['main_floor_area'] == ''), 'main_floor_area'] = data['state_to'].map(state_mean)


In [6]:
data


,id,state_to,district_to,mukim,section,scheme,address,lot_type,lot_no,status,...,title_no,sector,tenure,leasehold_period,property_type,floor_above,floor_below,main_floor_area,consider,duration_days
0,1940,Selangor,Petaling,Bandar Petaling Jaya Selatan,NaN,Tmn Medan,"NO 4, LORONG PJS 2D/15A",NO. PEJABAT TANAH,115,TUNGGAL,...,190821,RESIDENTIAL,LEASEHOLD,99.0,Terraced House,2.0,0.0,156.000000,50000.0,2830
1,9363,Johor,Batu Pahat,Tanjung Sembrong,NaN,Tmn Sri Aman,"10, JALAN KUNDANG",HAKMILIK MUKTAMAD,12828,TUNGGAL,...,491878,RESIDENTIAL,FREEHOLD,NaN,Terraced House,1.0,0.0,68.090000,107200.0,3054
2,9942,Selangor,Petaling,Bandar Sunway,NaN,Pangsapuri Suria Subang,"A-11-1, JLN PJS 10/1",NO. STRATA,1767(M1A/12/145),TUNGGAL,...,40578,RESIDENTIAL,LEASEHOLD,99.0,Condominium/Apartment,20.0,0.0,120.000000,540000.0,2870
3,11117,Penang,Seberang Perai Selatan,6,NaN,Tmn Widuri,"NO 39, LORONG WIDURI 29",HAKMILIK MUKTAMAD,2184,TUNGGAL,...,133637,RESIDENTIAL,FREEHOLD,NaN,Terraced House,1.0,0.0,52.440000,55000.0,2773
4,11636,Selangor,Gombak,Bandar Batu Arang,2,Kota Putri Bandar Baru Bt Arang,"6, JALAN KOTA PUTERI 5/5G",NO. PEMAJU,180-NO.6,TUNGGAL,...,4/003762/2015,RESIDENTIAL,LEASEHOLD,99.0,Terraced House,2.0,0.0,134.980000,355000.0,2830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600681,33654102,Penang,Seberang Perai Utara,5,NaN,Taman Permatang Sintok,"B-2-18, LORONG SINTOK 2",NO. STRATA,5204(M1/3/60),LOT TUNGGAL,...,88240,RESIDENTIAL,FREEHOLD,NaN,Flat,5.0,0.0,108.871596,200000.0,148
600682,33654103,Penang,Seberang Perai Tengah,6,NaN,Tmn Nagasari,"22, LORONG NAGASARI 8",HAKMILIK MUKTAMAD,2756,LOT TUNGGAL,...,2151,RESIDENTIAL,FREEHOLD,NaN,Terraced House,2.0,0.0,107.800250,250000.0,157
600683,33654104,Penang,Seberang Perai Selatan,14,NaN,Tmn Tambun Indah,"3-20, LORONG TAMBUN INDAH 11",NO. STRATA,5082(M1/4/99),LOT TUNGGAL,...,62720,RESIDENTIAL,FREEHOLD,NaN,Flat,5.0,0.0,113.156935,80000.0,169
600684,33654107,Penang,Seberang Perai Utara,12,NaN,Tmn Tasek Rethina,"G-09, BLOK A, JALAN ARA KUDA",NO. STRATA,3434(M1/1/9),LOT TUNGGAL,...,458,RESIDENTIAL,FREEHOLD,NaN,Flat,5.0,0.0,108.871596,100000.0,146


In [7]:
# Check for missing values
missing_values = data.isnull().sum()

# Print the number of missing values in each column
print(missing_values)

id                       0
state_to                 0
district_to           1418
mukim                 1340
section             526990
scheme                1188
address                 48
lot_type                16
lot_no                   5
status               49234
transaction_date         0
landarea                 0
unit                     5
title_type              15
title_no                36
sector                   0
tenure                   0
leasehold_period    414450
property_type         3967
floor_above           2118
floor_below          49234
main_floor_area          0
consider                 0
duration_days            0
dtype: int64


In [8]:
# create label encoder object
le = LabelEncoder()

# create separate label encoder objects for each categorical variable
state_to_encoder = LabelEncoder()
district_to_encoder = LabelEncoder()
mukim_encoder = LabelEncoder()
scheme_encoder = LabelEncoder()
tenure_encoder = LabelEncoder()
status_encoder = LabelEncoder()
property_type_encoder = LabelEncoder()


# apply label encoding to each categorical column separately
data['state_to_encoded'] = state_to_encoder.fit_transform(data['state_to'])
data['district_to_encoded'] = district_to_encoder.fit_transform(data['district_to'])
data['mukim_encoded'] = mukim_encoder.fit_transform(data['mukim'])
data['scheme_encoded'] = scheme_encoder.fit_transform(data['scheme'])
data['tenure_encoded'] = tenure_encoder.fit_transform(data['tenure'])
data['status_encoded'] = status_encoder.fit_transform(data['status'])
data['property_type_encoded'] = property_type_encoder.fit_transform(data['property_type'])




# select numerical features
num_features = data.select_dtypes(exclude='object')

# Define input variable names
input_vars = ["state_to_encoded", "district_to_encoded", "mukim_encoded", "scheme_encoded", "tenure_encoded","status_encoded", "property_type_encoded", "main_floor_area", "floor_above", "landarea", "duration_days"]

# Define X and y
X = data[input_vars].values
y = data['consider'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an XGBoost model
model = XGBRegressor()
model.fit(X_train, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [9]:
# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print("Training score:", train_score)




Training score: 0.999999972907259


In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Calculate the range of R-squared values
y_mean = np.mean(y_test)
ss_total = np.sum((y_test - y_mean) ** 2)
ss_residual = np.sum((y_test - y_pred) ** 2)
r2_range = 1 - (ss_residual / ss_total)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("R-squared:", r2)
print("R-squared Range:", r2_range)


RMSE: 1184309.005002668
MSE: 1402587819330.4094
MAE: 154823.43953230977
R-squared: -0.00984540006721879
R-squared Range: -0.00984540006721879


In [ ]:
# Create the directory if it doesn't exist
if not os.path.exists(r'C:\Users\dell\Desktop\DataPrime\model_dataprime'):
    os.makedirs(r'C:\Users\dell\Desktop\DataPrime\model_dataprime')

# Save the trained model to a file using pickle
with open(r'C:\Users\dell\Desktop\DataPrime\model_dataprime\xgb_model_dataprime.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the label encoder objects to a file using pickle
label_encoders = {'state_to': state_to_encoder, 
                  'district_to': district_to_encoder, 
                  'mukim': mukim_encoder, 
                  'scheme': scheme_encoder, 
                  'tenure': tenure_encoder, 
                  'status': status_encoder, 
                  'property_type': property_type_encoder
                 }





with open(r'C:\Users\dell\Desktop\DataPrime\model_dataprime\label_encoders_dataprime.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [ ]:
#Test 
# Load the label encoder objects from the file
with open('label_encoders_dataprime_deep.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Retrieve the label encoder for 'state_to'
state_to_encoder = label_encoders['state_to']

# Transform a new data point using the label encoder
new_state_to = 'Selangor'
new_state_to_encoded = state_to_encoder.transform([new_state_to])[0]

print(new_state_to_encoded)

In [ ]:
#Test 
# Load the label encoder objects from the file
with open('label_encoders_dataprime_deep.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Retrieve the label encoder for 'state_to'
state_to_encoder = label_encoders['state_to']

# Transform a new data point using the label encoder
new_state_to = 'Johor'
new_state_to_encoded = state_to_encoder.transform([new_state_to])[0]

print(new_state_to_encoded)